In [ ]:
import xml.etree.ElementTree as ET
import os
import cv2
from PIL import Image
import numpy as np
from pathlib import Path

In [ ]:
#Path("norway_640").mkdir(parents=True, exist_ok=True)
Path("norway_640/annotations/xmls/").mkdir(parents=True, exist_ok=True)
Path("norway_640/images/").mkdir(parents=True, exist_ok=True)

In [ ]:
norwayList = os.listdir('RDD2022_all_countries/Norway/train/annotations/xmls/')
sorted_norwayList = sorted(norwayList)

In [ ]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        if image.shape[0] - y < windowSize[1]*0.7:
            break
        for x in range(0, image.shape[1], stepSize):
            if image.shape[1] - x < windowSize[0]*0.7:
                break
            else:
                yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [ ]:
def _pretty_print(current, parent=None, index=-1, depth=0):
    for i, node in enumerate(current):
        _pretty_print(node, current, i, depth + 1)
    if parent is not None:
        if index == 0:
            parent.text = '\n' + ('\t' * depth)
        else:
            parent[index - 1].tail = '\n' + ('\t' * depth)
        if index == len(parent) - 1:
            current.tail = '\n' + ('\t' * (depth - 1))

In [ ]:
for nw in temp_List:
    imageName = 'RDD2022_all_countries/Norway/train/images/' + nw.replace('.xml', '.jpg')
    print(imageName)
    image = cv2.imread(imageName)
#    print(image.shape)
    features = []
    windows = sliding_window(image, 400, (640, 640))
#    windows = sliding_window(image, 400, (1024, 1024))
#    print(len(list(windows)))
    idx = 0
    for window in windows:
#        print(window)
#        print(window[2])
        data = window[2][:,:,::-1]
        img = Image.fromarray(data)
        img.save("./norway_640/train/images/" + imageName.split('/')[-1].replace('.jpg', '_') + str(idx) + ".jpg")
        idx += 1

In [ ]:
for nw in sorted_norwayList:
    print(nw)
    
    doc = ET.parse('RDD2022_all_countries/Norway/train/annotations/xmls/' + nw)
    root = doc.getroot()

    labelWindowSize = 640
    labelStepSize = 400

#     labelWindowSize = 1024
#     labelStepSize = 400


    for objects in root.findall('size'):
        width = int(objects.find('width').text)
        height = int(objects.find('height').text)
        #print(width, height)

    cnt = 0
    for y in range(0, height, labelStepSize):
        if height - y < labelWindowSize*0.7:
            break
    
        for x in range(0, width, labelStepSize):
            #print(x)
            #print(x, x+labelWindowSize, y, y+labelWindowSize)
            if width - x < labelWindowSize*0.7:
                break

            root2 = ET.Element("annotation")
            element1 = ET.Element("folder")
            element2 = ET.Element("filename")
            element3 = ET.Element("size")
            element4 = ET.Element("segmented")

            root2.append(element1)
            root2.append(element2)
            root2.append(element3)
            root2.append(element4)

            element1.text = 'images'
            element2.text = nw.split('.')[0] + '_' + str(cnt) + ".jpg"
            element4.text = '0'

            sub_element3 = ET.SubElement(element3, "width")
            sub_element4 = ET.SubElement(element3, "height")
            
            if width - x > labelWindowSize:
                sub_element3.text = str(labelWindowSize)
            else:
                sub_element3.text = str(width - x)
                

            if height - y > labelWindowSize:
                sub_element4.text = str(labelWindowSize)
            else:
                sub_element4.text = str(height - y)

            for objects in root.findall('object'):
                tree2 = ET.ElementTree(root2)
                name = str(objects.find('name').text)
                #print('name = ', name)

                if name not in ['D00', 'D10', 'D20', 'D40']:
                    continue

                for tmp in objects.findall('bndbox'):
                    xmin = int(float(tmp.find('xmin').text))
                    ymin = int(float(tmp.find('ymin').text))
                    xmax = int(float(tmp.find('xmax').text))
                    ymax = int(float(tmp.find('ymax').text))
                    #print(xmin, ymin, xmax, ymax)

                xmin = max(x, xmin)
                ymin = max(y, ymin)
                xmax = min(x+labelWindowSize, xmax)
                ymax = min(y+labelWindowSize, ymax)
                if xmin < xmax and ymin < ymax:
                    element5 = ET.Element("object")
                    sub_element6 = ET.SubElement(element5, "name")
                    sub_element6.text = name
                    sub_element7 = ET.SubElement(element5, "bndbox")
                    sub_sub_element7 = ET.SubElement(sub_element7, "xmin")
                    sub_sub_element7.text = str(xmin - x)
                    sub_sub_element7 = ET.SubElement(sub_element7, "ymin")
                    sub_sub_element7.text = str(ymin - y)
                    sub_sub_element7 = ET.SubElement(sub_element7, "xmax")
                    sub_sub_element7.text = str(xmax - x)
                    sub_sub_element7 = ET.SubElement(sub_element7, "ymax")
                    sub_sub_element7.text = str(ymax - y)
                    root2.append(element5)

            _pretty_print(root2)
            tree2 = ET.ElementTree(root2)
            fileName = "./norway_640/annotations/xmls/" + nw.split('.')[0] + '_' + str(cnt) + ".xml"
            cnt += 1
            with open(fileName, "wb") as file:
                tree2.write(file)